# Dynamic Time Warping

![DTW](https://databricks.com/wp-content/uploads/2019/04/Euclidean_vs_DTW.jpg)

<br>

[Image Credits](https://databricks.com/blog/2019/04/30/understanding-dynamic-time-warping.html)

<br>

Dynamic Time Warping (DTW) è un algoritmo per misurare la similarità tra due serie temporali che possono variare in velocità. Ad esempio, le somiglianze nella deambulazione potrebbero essere rilevate usando il DTW, anche se una persona cammina più velocemente dell'altra, o se ci fossero accelerazioni e decelerazioni nel corso di un'osservazione. E' molto utilizzata in ambito *IoT* ad esempio per il riconoscimento vocale perchè no risente se una persona parla più velocemente o più lentamente della voce campione preregistrata. Il DTW può essere applicato a sequenze temporali di dati video, audio e grafici, infatti, tutti i dati che possono essere trasformati in una sequenza lineare possono essere analizzati con DTW, ma in particolare è molto efficace con serie ad alta frequenza di campionamento.

Per saperne di più la pagina [Wikipedia](https://en.wikipedia.org/wiki/Dynamic_time_warping)

<br>

Date due sequenze vediamo come funziona il **DTW** e le sue possibili applicazioni.

In [ ]:
import numpy as np
serie_1 = np.array([1, 2, 3, 5, 5, 5, 6], ndmin = 2)
serie_2 = np.array([1, 1, 2, 2, 3, 5], ndmin =2)

Le due serie hanno lunghezza diversa quindi un confronto punto per punto con distanza euclidea (o altra distanza) non sarebbe del tutto corretto.
Supponiamo che le due serie partano nello stesso periodo e quindi la serie_2 finisca prima, quindi aggiungiamo un elemento per calcolare la distanza.

In [ ]:
serie_2New = serie_2.copy()
serie_2New = np.append(serie_2New, 0)

Calcoliamo la distanza euclidea

<br>

![euclidean](https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0)

<br>

[Image Credits](https://en.wikipedia.org/wiki/Euclidean_distance)

In [ ]:
distance = np.sqrt(np.sum((serie_1-serie_2New)**2))
distance

Per implementare il DTW prima di tutto abbiamo bisogno di una matrice *n x m* dove n ed m sono la dimensione delle serie più uno, vedremo di seguito perchè.

In [ ]:
n = serie_1.shape[1]+ 1
m = serie_2.shape[1]+ 1

In [ ]:
DTW = np.zeros([n,m])

Il primo elemento in alto a sinistra [0,0] deve essere pari a 0, mentre prima riga e prima colonna pari ad infinito, questo serve per il *cold start* dell'algoritmo, dovendo confrontare i periodi precedenti al primo passo non si incorre in situaziono anomale.

In [ ]:
DTW[0,1:] = np.inf
DTW[1:,0] = np.inf
print(DTW)

Viene impostata una variabile **w -> windows size** che indica la finestra temporale di riferimento, questa è il massimo tra un intero positivo scelto dal data scientist e la differenza tra le dimensioni della matrice. In questo caso inserisco *w = 1* .

In [ ]:
w = max([1, abs(n-m)])
w

Ora per ogni step, considereremo la distanza tra ogni punto e lo aggiungeremo con la distanza minima che abbiamo trovato finora. Questo ci darà la distanza ottimale di due sequenze fino a quel periodo.

```
DTW[i,j] = distance(i, j) + min(DTW[i-1, j], DTW[i, j-1], DTW[i-1, j-1])
```

La sequenza di minimi si chiama **warping path**.

<br>

![warping_path](https://i.stack.imgur.com/2Bfjj.jpg)

<br>

[Image Credits](https://riptutorial.com/algorithm/example/24981/introduction-to-dynamic-time-warping)

In [ ]:
for i in range(1,n):
    for j in range(max([1,i-w]), min([m, i+w])):
        dist = abs(serie_1[:,i-1] - serie_2[:,j-1]) 
        DTW[i,j] = dist + min([DTW[i-1,j], DTW[i,j-1], DTW[i-1,j-1]])

print(DTW)

Quindi la distanza tra le serie è 1. Come si può vedere la misura non è influenzata dalla diversa lunghezza delle serie, non bisogna usare artifici per avere una misura di similarità.

Questa implementazione è molto semplice ed ha complessità **O(m x n)** ma esistono altre versioni ottimizzate dell'algoritmo come il FastDTW.

Il Dynamic Time Warping è molto utile negli algoritmi basati sulle distanze, come ad esempio un *KNN* o *K-Means* sulle serie e non sul singolo punto, può anche essere utilizzato per valutare la precisione di una algoritmo di forecasting per le serie temporali.

<br>
<br>

***N.B. Di seguito una versione dell'algoritmo che non tiene conto del parametro w e restituisce la stessa matrice nell'immagine. Il valore finale non cambia***

In [ ]:
for i in range(1,n):
    for j in range(1, m):
        dist = abs(serie_1[:,i-1] - serie_2[:,j-1]) 
        DTW[i,j] = dist + min([DTW[i-1,j], DTW[i,j-1], DTW[i-1,j-1]])

print(DTW)